In [3]:
!pip install pandas sqlalchemy
!pip install langchain-community langchain langchain-openai openai pdfplumber pandas pypdf


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from IPython.display import clear_output
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import pandas as pd
import json
import os
import pickle

In [ ]:
# Definir el modelo LLM (asegúrate de tener tu API key configurada)
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=api_key)
# Crear un prompt para procesar tablas
prompt = PromptTemplate(
    input_variables=["pdf_content"],
    template=(
        "Extrae las tablas del siguiente texto del PDF y organízalas en un formato tabular JSON."
        "El texto es:\n{pdf_content}"
        "Solo devuelve el JSON, sin ```json\n"
    )
)
 
# Crear una cadena LangChain
chain = LLMChain(llm=llm, prompt=prompt)
 
# Directorio de entrada y archivo de salida
pdf_directory = "pdf"  # Cambia a la ruta de tu directorio con los PDFs
output_file = "output.json"  # Archivo de salida donde se almacenará el resultado
 
# Crear un diccionario para almacenar los resultados de cada archivo PDF
results = {}
 
# Iterar sobre los archivos PDF en el directorio "pdfs"
pdf_directory = "pdf"  # Reemplaza con la ruta correcta a tu directorio
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_directory, filename)
        print(f"Procesando archivo: {filename}")
 
        try:
          # Cargar y leer el PDF
          loader = PyPDFLoader(filepath)
          documents = loader.load()
 
          # Combinar texto del PDF
          pdf_text = " ".join([doc.page_content for doc in documents])
 
          # Ejecutar el modelo para procesar el contenido del PDF
          response = chain.run(pdf_content=pdf_text)
          #print(response) # Imprime la respuesta JSON para cada archivo
 
          # Guardar la respuesta JSON asociada al archivo
          results[filename] = json.loads(response)
          
          
        except Exception as e:
          print(f"Error al procesar {filename}: {e}")
          
        try:
                data = json.loads(response)

                # Adaptar la estructura del JSON a la deseada
                resultados_archivo = {
                    "nombre_archivo": filename,
                    "prescripcion_medico_veterinaria": data.get("prescripcion_medico_veterinaria", {}),
                    "unidades_cultivo": data.get("unidades_cultivo", []),
                    "cantidad_total_alimento": data.get("cantidad_total_alimento", {}),
                    "dieta": data.get("dieta", {})  # Si solo hay una dieta, se guarda como objeto
                }

                results[filename] = resultados_archivo

        except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON para {filename}: {e}")
        except Exception as e:
                print(f"Error al convertir a DataFrame para {filename}: {e}")
# Escribir los resultados en un archivo JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)
 
print(f"Procesamiento completado. Resultados guardados en '{output_file}'.")





In [2]:
import os
import json
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv

# Cargar variables del .env
load_dotenv()


# Definir el modelo LLM
api_key = os.getenv("API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=api_key)

# Crear el prompt para extraer tablas
prompt = PromptTemplate(
    input_variables=["pdf_content"],
    template=(
        "Extrae las tablas del siguiente texto del PDF y organízalas en un formato tabular JSON. "
        "Después normaliza el JSON extraído. "
        "El texto es:\n{pdf_content} "
        "Solo devuelve el JSON, sin ```json\n"
    )
)

# Crear cadena de procesamiento LangChain
chain = LLMChain(llm=llm, prompt=prompt)

# Ruta de entrada y salida
pdf_directory = "pdf"
output_dir = "inten"
os.makedirs(output_dir, exist_ok=True)

# Función para desanidar JSON de forma recursiva
def flatten_json(y, parent_key="", sep="."):
    """Desanida un JSON recursivamente y devuelve un diccionario plano"""
    items = []
    if isinstance(y, list):  # Si es lista, convertir en un diccionario enumerado
        y = {str(i): v for i, v in enumerate(y)}

    for k, v in y.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_json(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, elem in enumerate(v):
                if isinstance(elem, (dict, list)):
                    items.extend(flatten_json(elem, f"{new_key}[{i}]", sep=sep).items())
                else:
                    items.append((f"{new_key}[{i}]", elem))
        else:
            items.append((new_key, v))
    return dict(items)

# Procesar archivos PDF
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_directory, filename)
        print(f"📄 Procesando archivo: {filename}")
        
        try:
            loader = PyPDFLoader(filepath)
            documents = loader.load()

            # Combinar contenido de páginas
            pdf_text = " ".join([doc.page_content for doc in documents])

            # Obtener tablas desde el modelo LLM
            response = chain.run(pdf_content=pdf_text)

            try:
                data = json.loads(response)

                # 🔹 Si el JSON es una lista, convertirla en un diccionario antes de continuar
                if isinstance(data, list):
                    data = {str(i): v for i, v in enumerate(data)}

                flattened_data = flatten_json(data)

                # Convertir a DataFrame
                df_normalized = pd.DataFrame([flattened_data])

                # Nombre del archivo Excel para este PDF
                excel_filename = filename.replace(".pdf", ".xlsx")
                excel_filepath = os.path.join(output_dir, excel_filename)

                # Guardar en Excel
                with pd.ExcelWriter(excel_filepath, engine="openpyxl", mode="w") as writer:
                    df_normalized.to_excel(writer, sheet_name="Datos", index=False)

                print(f"✅ Datos de '{filename}' guardados en '{excel_filepath}'.")

            except json.JSONDecodeError as e:
                print(f"❌ Error al decodificar JSON para {filename}: {e}")

        except Exception as e:
            print(f"❌ Error al procesar {filename}: {e}")

print("✅ Procesamiento completado.")



📄 Procesando archivo: Agua Buena - ABR 2024.pdf
✅ Datos de 'Agua Buena - ABR 2024.pdf' guardados en 'inten\Agua Buena - ABR 2024.xlsx'.
📄 Procesando archivo: Agua Buena - Declaración_SINADER_folio_1000494 - DIC 2024.pdf
✅ Datos de 'Agua Buena - Declaración_SINADER_folio_1000494 - DIC 2024.pdf' guardados en 'inten\Agua Buena - Declaración_SINADER_folio_1000494 - DIC 2024.xlsx'.
📄 Procesando archivo: Agua Buena - Declaración_SINADER_folio_856717.pdf
✅ Datos de 'Agua Buena - Declaración_SINADER_folio_856717.pdf' guardados en 'inten\Agua Buena - Declaración_SINADER_folio_856717.xlsx'.
📄 Procesando archivo: Agua Buena - Declaración_SINADER_folio_868652 - FEB 2024.pdf
✅ Datos de 'Agua Buena - Declaración_SINADER_folio_868652 - FEB 2024.pdf' guardados en 'inten\Agua Buena - Declaración_SINADER_folio_868652 - FEB 2024.xlsx'.
📄 Procesando archivo: Agua Buena - Declaración_SINADER_folio_888389 - MAR 2024.pdf
✅ Datos de 'Agua Buena - Declaración_SINADER_folio_888389 - MAR 2024.pdf' guardados en '